In [11]:
!pip install tensorflow

In [12]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [13]:
actions = ['rabbit', 'butterfly', 'shark']

data = np.concatenate([
    np.load('dataset/seq_rabbit_1669853445.npy'),
    np.load('dataset/seq_butterfly_1669853445.npy'),
    np.load('dataset/seq_shark_1669853445.npy')
], axis=0)

data.shape

(3072, 30, 100)

In [14]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(3072, 30, 99)
(3072,)


In [15]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(3072, 3)

In [16]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(2764, 30, 99) (2764, 3)
(308, 30, 99) (308, 3)


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                41984     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 44,163
Trainable params: 44,163
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/200
85/87 [============================>.] - ETA: 0s - loss: 22.5173 - acc: 0.4346
Epoch 1: val_acc improved from -inf to 0.56818, saving model to models\model.h5
87/87 [==============================] - 5s 25ms/step - loss: 22.3487 - acc: 0.4363 - val_loss: 14.1859 - val_acc: 0.5682 - lr: 0.0010
Epoch 2/200
86/87 [============================>.] - ETA: 0s - loss: 6.5613 - acc: 0.5996
Epoch 2: val_acc improved from 0.56818 to 0.62987, saving model to models\model.h5
87/87 [==============================] - 2s 23ms/step - loss: 6.5592 - acc: 0.5995 - val_loss: 4.1946 - val_acc: 0.6299 - lr: 0.0010
Epoch 3/200
86/87 [============================>.] - ETA: 0s - loss: 3.0306 - acc: 0.6693
Epoch 3: val_acc improved from 0.62987 to 0.69481, saving model to models\model.h5
87/87 [==============================] - 2s 19ms/step - loss: 3.0207 - acc: 0.6704 - val_loss: 2.0512 - val_acc: 0.6948 - lr: 0.0010
Epoch 4/200
85/87 [============================>.] - ETA: 0s - loss: 1.2095 - acc:

86/87 [============================>.] - ETA: 0s - loss: 0.3729 - acc: 0.8347
Epoch 30: val_acc did not improve from 0.83766
87/87 [==============================] - 2s 18ms/step - loss: 0.3719 - acc: 0.8354 - val_loss: 0.3935 - val_acc: 0.8149 - lr: 0.0010
Epoch 31/200
84/87 [===========================>..] - ETA: 0s - loss: 0.3892 - acc: 0.8244
Epoch 31: val_acc did not improve from 0.83766
87/87 [==============================] - 1s 17ms/step - loss: 0.3874 - acc: 0.8245 - val_loss: 0.4157 - val_acc: 0.8149 - lr: 0.0010
Epoch 32/200
86/87 [============================>.] - ETA: 0s - loss: 0.3571 - acc: 0.8427
Epoch 32: val_acc did not improve from 0.83766
87/87 [==============================] - 1s 17ms/step - loss: 0.3562 - acc: 0.8433 - val_loss: 0.3802 - val_acc: 0.8247 - lr: 0.0010
Epoch 33/200
86/87 [============================>.] - ETA: 0s - loss: 0.3326 - acc: 0.8474
Epoch 33: val_acc improved from 0.83766 to 0.84740, saving model to models\model.h5
87/87 [==================

87/87 [==============================] - ETA: 0s - loss: 0.0716 - acc: 0.9725
Epoch 59: val_acc did not improve from 0.97403
87/87 [==============================] - 1s 17ms/step - loss: 0.0716 - acc: 0.9725 - val_loss: 0.0995 - val_acc: 0.9545 - lr: 0.0010
Epoch 60/200
86/87 [============================>.] - ETA: 0s - loss: 0.1648 - acc: 0.9622
Epoch 60: val_acc did not improve from 0.97403
87/87 [==============================] - 1s 17ms/step - loss: 0.1643 - acc: 0.9624 - val_loss: 0.0959 - val_acc: 0.9643 - lr: 0.0010
Epoch 61/200
84/87 [===========================>..] - ETA: 0s - loss: 0.0685 - acc: 0.9818
Epoch 61: val_acc improved from 0.97403 to 0.99351, saving model to models\model.h5
87/87 [==============================] - 1s 17ms/step - loss: 0.0680 - acc: 0.9815 - val_loss: 0.0372 - val_acc: 0.9935 - lr: 0.0010
Epoch 62/200
83/87 [===========================>..] - ETA: 0s - loss: 0.0642 - acc: 0.9812
Epoch 62: val_acc did not improve from 0.99351
87/87 [==================

86/87 [============================>.] - ETA: 0s - loss: 0.4605 - acc: 0.7133
Epoch 89: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4606 - acc: 0.7135 - val_loss: 0.5017 - val_acc: 0.6396 - lr: 0.0010
Epoch 90/200
83/87 [===========================>..] - ETA: 0s - loss: 0.4554 - acc: 0.7150
Epoch 90: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4561 - acc: 0.7153 - val_loss: 0.5622 - val_acc: 0.6234 - lr: 0.0010
Epoch 91/200
84/87 [===========================>..] - ETA: 0s - loss: 0.4577 - acc: 0.7124
Epoch 91: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 16ms/step - loss: 0.4584 - acc: 0.7145 - val_loss: 0.5103 - val_acc: 0.6364 - lr: 0.0010
Epoch 92/200
87/87 [==============================] - ETA: 0s - loss: 0.4551 - acc: 0.7153
Epoch 92: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 

Epoch 119/200
87/87 [==============================] - ETA: 0s - loss: 0.4294 - acc: 0.7192
Epoch 119: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4294 - acc: 0.7192 - val_loss: 0.4828 - val_acc: 0.6429 - lr: 5.0000e-04
Epoch 120/200
86/87 [============================>.] - ETA: 0s - loss: 0.4343 - acc: 0.7198
Epoch 120: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4350 - acc: 0.7189 - val_loss: 0.4858 - val_acc: 0.6429 - lr: 5.0000e-04
Epoch 121/200
85/87 [============================>.] - ETA: 0s - loss: 0.4283 - acc: 0.7176
Epoch 121: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4282 - acc: 0.7185 - val_loss: 0.5117 - val_acc: 0.6494 - lr: 5.0000e-04
Epoch 122/200
86/87 [============================>.] - ETA: 0s - loss: 0.4281 - acc: 0.7198
Epoch 122: val_acc did not improve from 0.99351
87/87 [======================

Epoch 149/200
85/87 [============================>.] - ETA: 0s - loss: 0.4213 - acc: 0.7213
Epoch 149: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4234 - acc: 0.7196 - val_loss: 0.4704 - val_acc: 0.6461 - lr: 5.0000e-04
Epoch 150/200
84/87 [===========================>..] - ETA: 0s - loss: 0.4378 - acc: 0.7210
Epoch 150: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4386 - acc: 0.7196 - val_loss: 0.4640 - val_acc: 0.6461 - lr: 5.0000e-04
Epoch 151/200
85/87 [============================>.] - ETA: 0s - loss: 0.4216 - acc: 0.7217
Epoch 151: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4225 - acc: 0.7214 - val_loss: 0.4799 - val_acc: 0.6494 - lr: 5.0000e-04
Epoch 152/200
85/87 [============================>.] - ETA: 0s - loss: 0.4203 - acc: 0.7210
Epoch 152: val_acc did not improve from 0.99351
87/87 [======================

85/87 [============================>.] - ETA: 0s - loss: 0.4155 - acc: 0.7213
Epoch 178: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4162 - acc: 0.7203 - val_loss: 0.4649 - val_acc: 0.6461 - lr: 2.5000e-04
Epoch 179/200
85/87 [============================>.] - ETA: 0s - loss: 0.4160 - acc: 0.7199
Epoch 179: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 17ms/step - loss: 0.4149 - acc: 0.7200 - val_loss: 0.4641 - val_acc: 0.6461 - lr: 2.5000e-04
Epoch 180/200
84/87 [===========================>..] - ETA: 0s - loss: 0.4140 - acc: 0.7206
Epoch 180: val_acc did not improve from 0.99351
87/87 [==============================] - 1s 16ms/step - loss: 0.4144 - acc: 0.7211 - val_loss: 0.5089 - val_acc: 0.6331 - lr: 2.5000e-04
Epoch 181/200
87/87 [==============================] - ETA: 0s - loss: 0.4131 - acc: 0.7221
Epoch 181: val_acc did not improve from 0.99351
87/87 [==============================] - 1s

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))